**Ciencia y analítica de datos**

Carlos Álvarez


Análisis de Componentes Principales (PCA)

---

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [ ]:
cereals_df = pd.read_csv('/content/drive/MyDrive/Equipo 28 Ciencia y Analítica de Datos/Actividad 7 PCA/Cereals.csv', encoding="utf-8-sig")
cereals_df

# **Parte 1**. EDA y preprocesamiento

1. Obtén estadísticas descriptivas para todas las variables del dataframe.

In [ ]:
# Considerar la variable shelf como variable categórica
cereals_df["shelf"] = cereals_df["shelf"].astype(str)
cereals_df["shelf"]

In [ ]:
# Extraemos las columnas numéricas del df
numeric_cols_names_list = [col  for col in cereals_df.columns if cereals_df[col].dtype in ['int64', 'float64']]
# Extraemos las columnas NO numéricas del df
non_numeric_cols_names_list = [col  for col in cereals_df.columns if cereals_df[col].dtype not in ['int64', 'float64']]
print(f"""
      El dataframe de cereales cuenta con:

            * COLUMNAS: {list(cereals_df.columns)}
            * TOTAL DE COLUMNAS: {len(cereals_df.columns)}

      De las cuales:

            * COLUMNAS NÚMERICAS: {numeric_cols_names_list}
            * COLUMNAS NÚMERICAS #: {len(numeric_cols_names_list)}

            * COLUMNAS NO NÚMERICAS: {non_numeric_cols_names_list}
            * COLUMNAS NO NÚMERICAS #: {len(non_numeric_cols_names_list)}
      """)

In [ ]:
# Calculamos el % de valores nulos por columna
nulls_df = pd.DataFrame((cereals_df.isna().sum() / cereals_df.shape[0]) * 100, columns=["Porcentaje de Valores Faltantes"])
nulls_df = nulls_df.join(pd.DataFrame(cereals_df.isna().sum(), columns=["Cantidad de Valores Faltantes"]))
nulls_df.sort_values(by=["Porcentaje de Valores Faltantes"], ascending=False, inplace=True)
nulls_df

In [ ]:
# Como el método "describe" no calcula el coeficiente de asimetría (Skewness) ni el coeficiente de Kurtosis
# Entonces los calculamos por separado

# Calculamos el coeficiente de asimetría
numeric_cols_skewness_coeff_df = pd.DataFrame(cereals_df[numeric_cols_names_list].skew(), columns=["skewness"])

# Calculamos el coeficiente de Kurtosis
numeric_cols_kurtosis_coeff_df = pd.DataFrame(cereals_df[numeric_cols_names_list].kurtosis(), columns=["kurtosis"])
# Ahora ejecutamos un join/merge para unir ambos coeficiente en un df
skewness_and_kurtosis_df = pd.merge(
    left=numeric_cols_skewness_coeff_df,
    right=numeric_cols_kurtosis_coeff_df,
    right_index=True,
    left_index=True
    )
# Ahora ejecutamos un join/merge para tener todas las estadisticas en una sola tabla
all_statistics_df = pd.merge(
    left=cereals_df.describe().T, # Tabla de estadísticas descriptivas comúnes
    right=skewness_and_kurtosis_df,
    right_index=True,
    left_index=True
)
# Finalmente miramos las estadisticas de todas las variables numéricas
all_statistics_df

In [ ]:
# Miramos las estadisticas de la variables no numéricas
cereals_df.describe(include="object").T

Genera algunos gráficos para familiarizarte con el conjunto de datos. Al menos deberás incluir los que te permitan responder los siguientes cuestionamientos:

**Nota.** Debes responder de manera explícita las preguntas, apoyándote de los resultados observados de cada gráfico.

2a) ¿Cuál es la frecuencia o conteo de cada categoría para las variables: `mfr` y `shelf` (por separado)?

Frecuencia de mfr (Fabricante):<br>
La mayoría de los cereales son fabricados por:
<br>K (Kellogg's)
<br>G (General Mills)
<br>P (Post)
<br><br>Los fabricantes con menos productos en el conjunto de datos son:
<br>A (American Home Food Products)
<br>N (Nabisco)
<br><br>Frecuencia de shelf (Estante):
<br>La mayoría de los cereales se encuentran en el estante 3.
<br>El estante 1 tiene la menor cantidad de cereales.
<br>El estante 2 tiene una cantidad intermedia de cereales.

In [ ]:
for cat_var in ["mfr", "shelf"]:
    plt.figure(figsize=(6, 3))
    ax = sns.countplot(data=cereals_df, x=cat_var)
    ax.tick_params(axis='x', labelsize=8)
    ax.set_title(f"Gráfico de barras para {cat_var}")

    for p in ax.patches:
        ax.annotate(
            f'{int(p.get_height())}',
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center',
            va='center',
            xytext=(0, 5),
            textcoords='offset points',
            fontsize=8
        )

    plt.show()

2b) Combina los resultados previos para observar, de cada fabricante, cuántos productos tiene en cada estante.

* **Kellogg's (K)** tiene una distribución bastante equilibrada de productos en los
tres estantes.<br>
* **General Mills (G)** tiene la mayoría de sus productos en el estante 3, seguido del estante 2.<br>
* **Post (P)** tiene más productos en el estante 3, con algunos productos en el estante 1 y 2.<br>
* Otros fabricantes como **American Home Food Products (A)**, **Nabisco (N)** y **Quaker  Oats (Q)** tienen una presencia menor, pero sus productos están distribuidos en varios estantes.

In [ ]:
# Gráfico de barras apiladas para observar de cada fabricante cuántos productos tiene en cada estante
plt.figure(figsize=(14, 8))
sns.countplot(x='mfr', hue='shelf', data=cereals_df)
plt.title('Número de productos por fabricante y estante')
plt.xlabel('Fabricante')
plt.ylabel('Conteo')
plt.legend(title='Estante')
plt.show()

 3a) ¿Cuál es el rango más frecuente de la variable `rating`?

 Del histograma de la distribución de la variable rating, se observa que el rango más frecuente de las calificaciones se encuentra aproximadamente entre 30 y 50. Este es el intervalo donde la mayoría de los cereales tienen sus calificaciones.

In [ ]:
# Crear un histograma para observar la distribución de la variable 'rating'
plt.figure(figsize=(10, 6))
sns.histplot(cereals_df['rating'], bins=10, kde=True)
plt.title('Distribución de la variable rating')
plt.xlabel('Rating')
plt.ylabel('Frecuencia')
plt.show()

3b) ¿Cuáles son los 10 cereales mejor evaluados y a qué fabricante corresponden?

In [ ]:
# Obtener los 10 cereales mejor evaluados
top_10_cereals = cereals_df.nlargest(10, 'rating')[['name', 'mfr', 'rating']].reset_index(drop=True)
top_10_cereals

4a) Compara la distribución de las calorías según el fabricante, por medio de boxplots.

In [ ]:
boxplot_variables = ["mfr", "calories"]
plt.figure(figsize=(8,4))
sns.boxplot(x=boxplot_variables[1], y=boxplot_variables[0], data=cereals_df)
plt.xlabel(boxplot_variables[1], fontsize=8)
plt.ylabel(boxplot_variables[0], fontsize=8)
plt.title(f'Boxplot\n{boxplot_variables[1]} por {boxplot_variables[0]}', fontsize=10)
plt.show()

**Kellogg's (K)**: Presenta una distribución de calorías bastante estrecha, con la mayoría de los productos entre 100 y 110 calorías aproximadamente. Además, cuenta con outliers que llegan hasta las 160 calorias o las 50 aprox.
<br>
**General Mills (G)**: Tiene una mayor dispersión en la cantidad de calorías, abarcando desde aproximadamente 110 calorías. El tamaño del boxplot nos indica que hay pocos datos.
<br>
**Post (P)**: Tiene una amplia dispersión de calorías, desde 100 hasta 120. Denotamos la ausencia de outliers.
<br>
**Nabisco (N)**: Los productos están cerca de los 80 y 90 calorías.
<br>
**Quaker Oats (Q)**: Sus productos tienden a estar en un rango más amplio, con la mayoría de ellos entre 70 y 120 calorías.
<br>
**Mypo (A)**: Hay muy pocos datos lo que hace que el boxplot sea práctimante un solo dato.
<br>
**Chex (R)**: Podemos ver que la concentración de la calorias es relativamente más alta que para las otras marcas, con valores desde los 100 hasta los 130 calorias.

4b) ¿Cuáles son las dos variables que tienen mayor correlación con `rating`?

Las dos variables con la correlación más "alta":
* sugars: -0.76
* calories: -0.69

In [ ]:
corr_df = cereals_df[numeric_cols_names_list].corr()

In [ ]:
# Para el tamaño del gráfico
plt.figure(figsize=(10, 8))

# Para crear un paleta de colores divergente
cmap = sns.diverging_palette(240, 10, as_cmap=True)

# Para le divergencia de los colores
vmin = -1 # Para que la diveregencia de los colores esten en función de un valor mínimo
vmax = 1 # Para que la diveregencia de los colores esten en función de un valor máximo

# Generamos el gráfico
sns.heatmap(corr_df,
            cmap=cmap, # La paleta de coores diveregente
            vmin=vmin, # El valor mínimo de diveregencia de color
            vmax=vmax, # El valor máximo de diveregencia de color
            xticklabels=corr_df.columns, # Las etiquetas del eje x
            yticklabels=corr_df.columns,# Las etiquetas del eje y
            center=0, annot=True, fmt=".2f", # Para añadir los valores a la matriz dibujada
            )
# Título del gráfico
plt.title(f'Matriz de Correlación Cereals Dataset', fontsize=12)
plt.show()

#La correlación positiva mayor viene dada por los valores de fibra(0.58) y la mas negativa viene dada por el contenido de azucares(-0.76).


5. Elimina todos los registros con algún dato faltante y reinicia el índice del dataframe para que quede con valores consecutivos.

In [ ]:
cereals_df.dropna(inplace=True)

In [ ]:
cereals_df.reset_index(drop=True, inplace=True)

# **Parte 2**. Ingeniería de características

6. Considerando que `rating` es la variable de salida, almacénala en una variable `y` y separa los predictores **numéricos** en `X`. Escala los valores de `X` y aplica `PCA` para proyectar los datos en el nuevo espacio de vectores.

In [ ]:
X = cereals_df[[x for x in numeric_cols_names_list if x != "rating"]]
y = cereals_df["rating"]

In [ ]:
#Escalando los valores de X:
#Primero se verifica que las distribuciones no esten sesgadas antes de proceder con el escalamiento.
#Se guardan aquellas variables sesgadas en una lista para su posterior tratamiento.
variables_sesgadas = []
variables_no_sesgadas = []
for i in all_statistics_df.index:
  print("El valor de asimetria para la variable {} es de:    {}".format(i, all_statistics_df.loc[i, "skewness"]))
  if( (all_statistics_df.loc[i, "skewness"] > 1) | (all_statistics_df.loc[i, "skewness"] < -1)):
    variables_sesgadas.append(i)
  else:
    variables_no_sesgadas.append(i)

In [ ]:
#Se comienza con la transformación o la normalización debido a que existen distribuciones asimetricas:
from sklearn.preprocessing import PowerTransformer
# Instanciamos el transformador
transformer = PowerTransformer(method="yeo-johnson", standardize=False)
# Ajustamos el transformador
transformer.fit(X[variables_sesgadas])
#Se aplica la transformación Yeo-Johnson a variables sesgadas
X_transf = pd.DataFrame(transformer.transform(X[variables_sesgadas]), columns= variables_sesgadas)
X_transf

In [ ]:
#Se vuelve a unir el dataframe con las variables no sesgadas
X_transf = X_transf.join(X[[x for x in variables_no_sesgadas if x != "rating"]])
X_transf

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_scaled = std_scaler.fit_transform(X_transf)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_projected = pca.fit_transform(X_scaled)
X_projected = pd.DataFrame(X_projected)

In [ ]:
X_projected

7. Obtén la curva del porcentaje de varianza acumulada y determine el número mínimo de componentes principales que explique más del 90% de la varianza. Imprime la información de dichos componentes.

* El número mínimo de componentes para explicar más del 90% de la varianza es 6.

In [ ]:
total_components = X.shape[1]

In [ ]:
sns.set_style('white')

plt.plot(np.cumsum(pca.explained_variance_ratio_)*100)
plt.title('PCA Analysis')
plt.xlabel('N-th Principal Component')
plt.ylabel('% Cumulative Variance Explained')
plt.xticks(np.arange(0,total_components,1))

labels = np.cumsum(pca.explained_variance_ratio_)*100
for i in range(total_components):
  plt.text(i,labels[i],str(format(labels[i],'.0f'))+'%')

In [ ]:
explained_cumulative_variance_array = np.cumsum(pca.explained_variance_ratio_)
n_components = np.argwhere(explained_cumulative_variance_array > 0.90)[0][0] + 1
n_components

8. ¿Cuáles son las tres variables más importantes en el cálculo del primer componente?

* fat
* protein
* weight

In [ ]:
pc_df = pd.DataFrame(abs(pca.components_[:n_components]), columns = X.columns, index=['Principal component {}'.format(i) for i in range(1, n_components + 1)])

In [ ]:
pc_df_pc1 = pc_df.T[["Principal component 1"]].sort_values(by=["Principal component 1"], ascending=False)

In [ ]:
ax = sns.barplot(
    x=pc_df_pc1["Principal component 1"],
    y=pc_df_pc1.index,
    hue=pc_df_pc1["Principal component 1"],
    palette="dark:blue_r",
    dodge=False,
    legend=False
)

for index, value in enumerate(pc_df_pc1["Principal component 1"]):
    ax.text(value, index, f'{value:.2f}', color='black', ha="left", va="center", fontsize=8)

plt.title("Importancia de caracteristicas para PC1")
plt.xlabel('Importancia')
plt.ylabel('Caracteristicas')
plt.xticks(rotation=90)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()

9. ¿Todas las variables categóricas son predictores? Aplica una técnica de encoding a las que sí lo sean. Justifica la elección de tu técnica.

* Recordando el describe de las variables categóricas, descartamos las variables "name", "type" como variables predictoras por las siguientes razones:
    * "name": Hay tantos registros únicos de name como renglones en el dataframe.
    * "type": Hay solo dos categorías de esta variable, pero una de ellas cuenta con un solo registro en la tabla.

In [ ]:
# Conteo de valores únicos para las variables categóricas
cereals_df.describe(include="object").T

In [ ]:
# Hay solo 1 registro del tipo "H" en la variable "type"
cereals_df["type"].value_counts()

In [ ]:
#Realizando gráficas de plots para las variables categóricas incluyendo a shelf.
fig, axes = plt.subplots(2,1, figsize=(10,8))
axes = axes.ravel()
for col, ax in zip(['mfr', 'shelf' ], axes):
  sns.boxplot(data = cereals_df, y = cereals_df[col], x=cereals_df['rating'], ax=ax)
  ax.set(title=f'{col}', xlabel=None)

#Observando las gráficas de boxplot para
  * mfr:
    * Se observa que hay fabricantes que cubren todo el rango del conjunto de otros fabricantes como lo es Q.
    * Sin embargo hay un rango que no es cubierto por Q y solo tiene N, de modo que cuando el fabricante es N
    * existirá un cambio en el rating. Por lo que se considera a N como una variable predictora.

  * shelf:
    * Los rangos de 1 y 3 estan traslapados y son diferentes al rango de 2.
    * Se puede considerar como predictora ya que si el numero de shelf es 2 el rango de rating estará por debajo de 40.



In [ ]:
# Usamos OneHotEncoder para codificar la variable "mfr" pues esta variable es categórico NO ordinal
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Instanciamos el encoder
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
# Ajustamos y transformamos los datos
mfr_onehot_df = pd.DataFrame(onehot_encoder.fit_transform(cereals_df[["mfr", "shelf"]]))
mfr_onehot_df.columns = list(onehot_encoder.get_feature_names_out())

10. Conjunta, en un dataframe, las valores proyectados en los componentes seleccionados (mínimo), las transformaciones obtenidas de las variables categóricas que serán predictores y la variable de salida. Almacena el dataframe resultante en archivo.

In [ ]:
final_df = X_projected.iloc[:,0:n_components]
final_df.columns = ['PC_{}'.format(i) for i in range(1, n_components + 1)]
final_df = pd.concat([final_df, mfr_onehot_df, y], axis=1)
final_df

In [ ]:
# Guardamos la tabla en un archivo csv
final_df.to_csv('/content/drive/MyDrive/Equipo 28 Ciencia y Analítica de Datos/Actividad 7 PCA/cereals_afterPCA_Equipo28.csv', index=False, encoding="utf-8-sig")